In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
df = pd.read_csv('train.csv',index_col='Id')

In [3]:
def location(arg):
    state = arg[0]
    country = arg[1]
    loc = str(state)+str(country)
    
    return loc

df['location'] = df[['Province_State','Country_Region']].apply(location,axis=1)
df.drop(['Province_State','Country_Region'],axis=1,inplace=True)

In [4]:
locations = df['location'].unique().tolist()

In [5]:
from sklearn.preprocessing import MinMaxScaler
from keras.preprocessing.sequence import TimeseriesGenerator
from keras.models import Sequential
from keras.layers import Dense,LSTM
from tqdm import tqdm

Using TensorFlow backend.


In [8]:
Res = None
ctr = 0
for i in tqdm(locations):
    df_train = df[df['location']==i].drop('location',axis=1)
    df_train['Date'] = pd.to_datetime(df_train['Date'],format='%d-%m-%Y')
    df_train.index = df_train['Date']
    df_train.drop('Date',inplace=True,axis=1)
    df_train.index.freq = 'D'
    
    Res = pd.concat([Res,df_train.iloc[-13:]],axis=0)
    train = df_train.copy()
    
    scaler = MinMaxScaler()
    scaler.fit(train)
    new_train = scaler.transform(train)
    length = 12
    batch = 2
    generator = TimeseriesGenerator(new_train,new_train,length=length,batch_size=batch)
    
    model = Sequential()
    model.add(LSTM(100,input_shape=(length,new_train.shape[1])))
    model.add(Dense(new_train.shape[1]))
    model.compile(optimizer='adam',loss='mse')
    model.fit_generator(generator,epochs=30)
    
    first_eval_batch = new_train[-length:]
    first_eval_batch = first_eval_batch.reshape((1, length, new_train.shape[1]))
    n_features = new_train.shape[1]
    test_predictions = []

    first_eval_batch = new_train[-length:]
    current_batch = first_eval_batch.reshape((1, length, n_features)) 

    for i in range(30):
        current_pred = model.predict(current_batch)[0]
        test_predictions.append(current_pred) 
        current_batch = np.append(current_batch[:,1:,:],[[current_pred]],axis=1)
        
    pred = scaler.inverse_transform(test_predictions)
    cases = [i[0] for i in pred]
    death = [i[1] for i in pred]
    
    new = pd.DataFrame(cases,death,columns=['Fatalities']).reset_index()
    new.columns = ['Fatalities','ConfirmedCases']
    
    Res = pd.concat([Res,new],axis=0)
    Res = Res.reset_index(drop=True)



  0%|                                                                                            | 0/3 [00:00<?, ?it/s]

Epoch 1/30
36/36 [==============================] - ETA: 10s - loss: 4.4321e- - ETA: 1s - loss: 0.0408     - ETA: 0s - loss: 0.040 - ETA: 0s - loss: 0.036 - ETA: 0s - loss: 0.033 - ETA: 0s - loss: 0.028 - 1s 17ms/step - loss: 0.0274
Epoch 2/30
36/36 [==============================] - ETA: 0s - loss: 0.001 - ETA: 0s - loss: 0.006 - ETA: 0s - loss: 0.006 - ETA: 0s - loss: 0.006 - 0s 6ms/step - loss: 0.0059
Epoch 3/30
36/36 [==============================] - ETA: 0s - loss: 0.015 - ETA: 0s - loss: 0.009 - ETA: 0s - loss: 0.006 - ETA: 0s - loss: 0.009 - ETA: 0s - loss: 0.007 - 0s 8ms/step - loss: 0.0069
Epoch 4/30
36/36 [==============================] - ETA: 0s - loss: 0.006 - ETA: 0s - loss: 0.008 - ETA: 0s - loss: 0.004 - ETA: 0s - loss: 0.004 - ETA: 0s - loss: 0.004 - 0s 7ms/step - loss: 0.0039
Epoch 5/30
36/36 [==============================] - ETA: 0s - loss: 9.4079e-0 - ETA: 0s - loss: 0.0012    - ETA: 0s - loss: 0.002 - ETA: 0s - loss: 0.002 - ETA: 0s - loss: 0.002 - 0s 8ms/step - 

C:\Users\pc\Anaconda3\lib\site-packages\ipykernel_launcher.py:46: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.


 33%|████████████████████████████                                                        | 1/3 [00:10<00:20, 10.18s/it]

Epoch 1/30
36/36 [==============================] - ETA: 10s - loss: 0.0000e+ - ETA: 1s - loss: 9.9148e-04 - ETA: 0s - loss: 0.0201    - ETA: 0s - loss: 0.041 - ETA: 0s - loss: 0.034 - ETA: 0s - loss: 0.031 - 1s 16ms/step - loss: 0.0304
Epoch 2/30
36/36 [==============================] - ETA: 0s - loss: 0.015 - ETA: 0s - loss: 0.008 - ETA: 0s - loss: 0.009 - ETA: 0s - loss: 0.009 - ETA: 0s - loss: 0.008 - 0s 7ms/step - loss: 0.0085
Epoch 3/30
36/36 [==============================] - ETA: 0s - loss: 0.001 - ETA: 0s - loss: 0.001 - ETA: 0s - loss: 0.002 - ETA: 0s - loss: 0.003 - ETA: 0s - loss: 0.004 - 0s 7ms/step - loss: 0.0040
Epoch 4/30
36/36 [==============================] - ETA: 0s - loss: 0.019 - ETA: 0s - loss: 0.036 - ETA: 0s - loss: 0.029 - ETA: 0s - loss: 0.020 - ETA: 0s - loss: 0.015 - 0s 8ms/step - loss: 0.0138
Epoch 5/30
36/36 [==============================] - ETA: 0s - loss: 0.004 - ETA: 0s - loss: 0.001 - ETA: 0s - loss: 7.2316e-0 - ETA: 0s - loss: 0.0014    - ETA: 0s - 

C:\Users\pc\Anaconda3\lib\site-packages\ipykernel_launcher.py:46: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.


 67%|████████████████████████████████████████████████████████                            | 2/3 [00:20<00:10, 10.15s/it]

Epoch 1/30
36/36 [==============================] - ETA: 14s - loss: 0.0000e+ - ETA: 1s - loss: 0.1225     - ETA: 0s - loss: 0.066 - ETA: 0s - loss: 0.089 - ETA: 0s - loss: 0.072 - ETA: 0s - loss: 0.060 - 1s 19ms/step - loss: 0.0586
Epoch 2/30
36/36 [==============================] - ETA: 0s - loss: 0.019 - ETA: 0s - loss: 0.016 - ETA: 0s - loss: 0.012 - ETA: 0s - loss: 0.014 - ETA: 0s - loss: 0.012 - 0s 7ms/step - loss: 0.0109
Epoch 3/30
36/36 [==============================] - ETA: 0s - loss: 0.001 - ETA: 0s - loss: 0.005 - ETA: 0s - loss: 0.003 - ETA: 0s - loss: 0.005 - ETA: 0s - loss: 0.005 - 0s 8ms/step - loss: 0.0052
Epoch 4/30
36/36 [==============================] - ETA: 0s - loss: 0.030 - ETA: 0s - loss: 0.004 - ETA: 0s - loss: 0.010 - ETA: 0s - loss: 0.008 - ETA: 0s - loss: 0.008 - 0s 7ms/step - loss: 0.0075
Epoch 5/30
36/36 [==============================] - ETA: 0s - loss: 0.030 - ETA: 0s - loss: 0.007 - ETA: 0s - loss: 0.006 - ETA: 0s - loss: 0.004 - ETA: 0s - loss: 0.004 

C:\Users\pc\Anaconda3\lib\site-packages\ipykernel_launcher.py:46: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.


100%|████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:30<00:00, 10.29s/it]

In [6]:
Res[['ConfirmedCases','Fatalities']].to_csv("submit.csv")